In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%reload_ext rpy2.ipython

In [79]:
import numpy as np
import pandas as pd
import scipy.stats as stats

from kaggle.house_prices import helpers
from kaggle.house_prices import missing
from kaggle.house_prices import outliers

combined_dataset = helpers.load_data()
combined_dataset = missing.fix_all(combined_dataset)
combined_dataset = outliers.remove_outliers(combined_dataset)
combined_dataset = combined_dataset.drop(columns=['Id'])
combined_dataset.shape

(2917, 81)

In [29]:
trans = {
    'log':    lambda x: np.log(x + 1),
    'sqrt':   lambda x: np.sqrt(x),
    'invcube': lambda x: x**(1/3)
}

In [35]:
df1 = (
    pd.melt(
        frame=(
            combined_dataset
            .drop(columns=['SalePrice'])
            .select_dtypes(include=[np.number])
        ), 
        var_name='var', 
        value_name='x'
    )
    .dropna(subset=['x'])
)

df1.query("var == 'LotFrontage'").head()
len(df1)

for tran_name, tran_fn in trans.items():
    df1[tran_name] = tran_fn(df1['x'])
    
df1.query("var == 'LotFrontage'").head()

,var,x
55423,LotFrontage,65.0
55424,LotFrontage,80.0
55425,LotFrontage,68.0
55426,LotFrontage,60.0
55427,LotFrontage,84.0


99178

,var,x,log,sqrt,invcube
55423,LotFrontage,65.0,4.189655,8.062258,4.020726
55424,LotFrontage,80.0,4.394449,8.944272,4.308869
55425,LotFrontage,68.0,4.234107,8.246211,4.081655
55426,LotFrontage,60.0,4.110874,7.745967,3.914868
55427,LotFrontage,84.0,4.442651,9.165151,4.379519


In [44]:
df2 = (
    pd.melt(
        frame=df1,
        id_vars=['var'],
        var_name='tran', 
        value_name='value'
    )
    .assign(
        value_normed = lambda df: (
            df
            .groupby(['var', 'tran'])
            ['value']
            .transform(lambda x: (x - x.mean()) / x.std())
        )
    )
    .drop(columns=['value'])
)

df2.query("var == 'LotArea'").groupby(['var', 'tran']).head(2)

,var,tran,value_normed
52506,LotArea,x,-0.216400
52507,LotArea,x,-0.069097
151684,LotArea,log,-0.101744
151685,LotArea,log,0.149410
250862,LotArea,sqrt,-0.210815
250863,LotArea,sqrt,0.024445
350040,LotArea,invcube,-0.182045
350041,LotArea,invcube,0.067009


In [63]:
df3 = (
    df2
    .groupby(['var', 'tran', 'value_normed'])
    .size()
    .to_frame('k')
)

df3.query("var == 'YrSold'")

k
var    tran    value_normed     
YrSold invcube -1.363338     619
               -0.602723     691
                0.157640     621
                0.917750     647
                1.677608     339
       log     -1.363451     619
               -0.602662     691
                0.157748     621
                0.917780     647
                1.677434     339
       sqrt    -1.363281     619
               -0.602753     691
                0.157585     621
                0.917735     647
                1.677695     339
       x       -1.363111     619
               -0.602844     691
                0.157422     621
                0.917689     647
                1.677956     339

In [74]:
scipy.stats.norm.cdf?

Object `scipy.stats.norm.cdf` not found.


In [80]:
(
    df3
    .assign(
        empirical=lambda df: df.groupby(['var', 'tran'])['k'].cumsum(),
        theoretical=lambda df: stats.norm.cdf(df['value_normed'])
    )
)

KeyError: 'value_normed'